In [24]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal
import sounddevice as sd

In [25]:
def load_audio_file(file_path):
    return librosa.load(file_path, sr=44100)

In [26]:
def correlation(x, y):
    return np.correlate(x, y, mode='full')

In [37]:
def energy(x):
    return np.sum(x**2)

In [29]:
def scale_signal(signal, factor):
    return factor * signal

In [30]:
def audio_authentication(audio_password, test_audio_password, threshold=0.9):
    min_length = min(len(audio_password), len(test_audio_password))
    r = np.corrcoef(audio_password[:min_length], test_audio_password[:min_length])[0, 1]
    return r > threshold

In [31]:
# Auto correlation of input signal
def auto_correlation(signal):
    y = correlation(signal, signal)
    is_even = np.all(y % 2 == 0)
    energy_y = energy(y)
    significance_y_0 = y[len(y) // 2]
    return y, is_even, energy_y, significance_y_0

In [32]:
# Auto correlation of delayed input signal
def delayed_auto_correlation(signal):
    p = correlation(signal[:-1], signal[:-1])
    return p

In [33]:
# Cross correlation of input signal and delayed input signal
def cross_correlation(signal1, signal2):
    q = correlation(signal1, signal2)
    return q

In [34]:
# Cross correlation of input signal and scaled input signal
def scaled_cross_correlation(signal1, scale_factor):
    s = correlation(signal1, scale_signal(signal1, scale_factor))
    return s

In [44]:
# Step 1: load both audio files
audio_password, _ = load_audio_file('audio_password.wav')
test_audio_password, _ = load_audio_file('audio_password.wav')

In [45]:
# 1. Auto correlation of input signal
y, is_even, energy_y, significance_y_0 = auto_correlation(audio_password)
print("Auto-correlation of input signal:")
print("Is Even:", is_even)
print("Energy:", energy_y)
print("Significance at y[0]:", significance_y_0)

Auto-correlation of input signal:
Is Even: False
Energy: 221393970.0
Significance at y[0]: 1053.6848


In [46]:
# 2. Auto correlation of delayed input signal
p = delayed_auto_correlation(audio_password)
print("Delayed Auto-correlation of input signal:", p)

Delayed Auto-correlation of input signal: [0. 0. 0. ... 0. 0. 0.]


In [47]:
# 3. Cross correlation of input signal and delayed input signal
q = cross_correlation(audio_password, np.roll(audio_password, 1))
print("Cross-correlation of input signal and delayed input signal:", q)

Cross-correlation of input signal and delayed input signal: [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.0070078e-06
 -1.3054814e-06  9.2410482e-07]


In [48]:
# 4. Cross correlation of input signal and scaled input signal
a = 2.0
s = scaled_cross_correlation(audio_password, a)
print("Cross-correlation of input signal and scaled input signal with a =", a)

Cross-correlation of input signal and scaled input signal with a = 2.0


In [49]:
# Audio Authentication Example
authenticated = audio_authentication(audio_password, test_audio_password)
print("\nAudio Authentication Result:", authenticated)


Audio Authentication Result: True
